![](./cover.png "")

# Intro Notebooks con F#

Tenemos varias opciones, como escribir codígo o usar Markdown para escribir nuestros articulos y despues mostrar nuestros resultados gracias a la ejecución del codígo.
En este Workshop trabajaremos el caso de nuestro cliente Tailwind Sports.

Así que lo primero que haremos sera cargar nuestras librerías a este espacio de trabajo.

## Trabajando con Nuget Packages desde un Notebook

Para usar las librerias de nuget haremos uso de la siguiente sintaxis : 

```
#r "<nuget package>, <nuget package version>"
```

En este proyecto utilizaremos las sig. librerías:


1. FSharp.Data : Nos permitira trabajar con servicios externos de manera facil.
2. Deedle : Inspirada en pandas, nos permitira trabajar con Dataframes.
3. Plotly.NET : Nos permite crear graficos en HTML muy similar a lo que hace Bokeh y Matplotlib en Python.
4. Newtonsoft.Json : Libreria para trabajar con JSON 

In [19]:
#r "nuget:FSharp.Data"
#r "nuget:Deedle"
#r "nuget: Plotly.NET, 2.0.0-preview.14"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.14"
#r "nuget: Newtonsoft.Json"

# Cargando datos en un Dataframe con Deedle

### Para cargar datos desde un archivo, usaremos Deedle, importamos el namespace de Deedle, y posteriormente usamos la siguiente sintaxis:

```
using Deedle;

var df = Deedle.Frame.ReadCsv("archivo.csv", separators: ",", hasHeaders: true);

# hasHeaders en caso de que el archivo CSV cuente con headers en la primera fila
# separators indica el tipo de separadores usados en el CSV, comunmente veremos ",", ";" , "\t" , etc 
```

En este taller usaremos el archivo `tailwind_sports_db.csv` que se encuentra en el directorio de DATASET

In [4]:
open Deedle
open System
open System.Globalization

let df = Frame.ReadCsv(@"./DATASET/tailwind_sports_db.csv", separators=",", hasHeaders= true)

#### Para visualizar los primeros 5 registros del dataframe, debemos usar la propiedad `Rows` y su metodo `Between`. Para imprimir usamos el metodo `Print` al final.


dataframe.<b style="color: purple;">Rows</b>.<b style="color: blue;">Between</b>(<b style="color: green;">indice 1</b>, <b style="color: orange;">indice 2</b>).**Print**()

In [6]:
df.Rows.Between(0, 5).Print()

0 -> series [ Orden => 1; Fecha => 06/08/2017; Medio => Propio; Vendedor => Directo en Tienda; Plataforma => Website;  ... ; Ingreso => 40.00] 
1 -> series [ Orden => 1; Fecha => 06/08/2017; Medio => Propio; Vendedor => Directo en Tienda; Plataforma => Website;  ... ; Ingreso => 50.00] 
2 -> series [ Orden => 1; Fecha => 06/08/2017; Medio => Propio; Vendedor => Directo en Tienda; Plataforma => Website;  ... ; Ingreso => 36.00] 
3 -> series [ Orden => 1; Fecha => 06/08/2017; Medio => Propio; Vendedor => Directo en Tienda; Plataforma => Website;  ... ; Ingreso => 40.00] 
4 -> series [ Orden => 1; Fecha => 06/08/2017; Medio => Propio; Vendedor => Directo en Tienda; Plataforma => Website;  ... ; Ingreso => 50.00] 
5 -> series [ Orden => 1; Fecha => 06/08/2017; Medio => Propio; Vendedor => Directo en Tienda; Plataforma => Website;  ... ; Ingreso => 45.00] 



#### Para visualizar el listado de columnas que tiene nuestro Dataframe, usamos la propiedad `Columns` y su propiedad `Keys`. Usando el comodin especial `display` podemos visualizar el resultado en el notebook.


<b style="color: orange;">display</b><b style="color: green;">(</b>**dataframe**.<b style="color: purple;">Columns</b>.<b style="color: blue;">Keys</b><b style="color: green;">)</b>

In [5]:
// Ver Columnas de nuestro Dataframe
display(df.Columns.Keys)

index,value
0,Orden
1,Fecha
2,Medio
3,Vendedor
4,Plataforma
5,Comisión
6,Tipo Orden
7,Tipo de Cliente
8,Sexo
9,Categoría


In [7]:
// Ver tipo de datos de las columnas
display(df.ColumnTypes)

index,value
0,System.Int32
1,System.String
2,System.String
3,System.String
4,System.String
5,System.Int32
6,System.String
7,System.String
8,System.String
9,System.String


# Ventas Totales

De acuerdo al documento, Tailwind Sports desea conocer el total de las ventas, para ello debemos hacer la suma de nuestra columna `Ingreso` y aplicar la operacion suma. Así que para seleccionar una columna en especifico de un dataframe, usaremos la siguiente sintaxis:

<b style="color: blue;">df</b>[<b style="color: orange;">columna</b>];

Para aplicar la suma podemos la propiedad `Values`, el cual nos va regresar un Enumerable del tipo de dato, y podemos aplicar `Sum` de `System.Linq` y obtener el resultado. Pero para evitar hacer todo eso, usaremos la clase `Stats` y su metodo `sum`, al cual le pasamos la Serie seleccionada. Ejemplo:

<b style="color: green;">Stats</b>.**sum**(<b style="color: blue;">df</b>[<b style="color: orange;">columna</b>]);

In [8]:
let x = df.GetColumn("Ingreso") |> Stats.sum
let vTotal = x.ToString "C"
display($"Ventas Totales: {vTotal}")

Ventas Totales: $2,571,850.00

# Integracion de las ventas totales

Si **$2,571,850.00** representa las ventas totales, queremos ver como se divide ese total entre compras y devoluciones. Para ello ocupamos filtrar por el `Tipo de Orden` . Para ello usaremos el metodo `FilterRowsBy` el cual recibe `columna a filtrar` y `valor a filtrar`.

var df_filtrado = dataframe.<b style="color: purple;">FilterRowsBy</b>(<b style="color: blue;">"columna"</b>, <b style="color: green;">"Valor"</b>)

var serieFiltrada = dataframe.<b style="color: purple;">FilterRowsBy</b>(<b style="color: blue;">"columna"</b>, <b style="color: green;">"Valor"</b>).[<b style="color: orange;">"Columna"</b>]

In [9]:
let compras = df.FilterRowsBy("Tipo Orden", "Compra").["Ingreso"] |> Stats.sum
let percCompras = compras / x
let devoluciones = df.FilterRowsBy("Tipo Orden", "Devolucion").["Ingreso"] |> Stats.sum
let percDevoluciones = devoluciones / x

type IntegracionVentasPerc(compra, pcompra, devolucion, pdevolucion, total) =
    member this.Compra = compra
    member this.PercCompra = pcompra
    member this.Devoluciones = devolucion
    member this.PercDevoluciones = pdevolucion
    member this.Total = total

let integracion = IntegracionVentasPerc(compras.ToString("C"), percCompras.ToString("0.00%"), devoluciones.ToString("C"), percDevoluciones.ToString("0.00%"), x.ToString("C"))

display(integracion)

Compra,PercCompra,Devoluciones,PercDevoluciones,Total
"$2,340,276.00",91.00%,"$231,574.00",9.00%,"$2,571,850.00"


# Graficar las ventas mensuales

En este apartado procederemos a usar Plotly. Para ello debemos importar el namespace Plotly.NET , el cuál contiene todas las funciones necesarias para renderizar nuestras grafícas. A continuacion mostraremos la sintaxis basíca para una grafica de columnas o barras.

```fsharp

open Plotly.NET
open Plotly.NET.LayoutObjects
open Plotly.NET.TraceObjects

// valores
let x =  [|10; 20; 30; 40; 50 |]
let y = [| "A"; "B"; "C"; "D"; "E" |];


// Le damos mas estilo
let mirroredXAxis =
    LinearAxis.init(
        Title = Title.init(Text="Texto Eje X"),
        ShowLine = true,
        Mirror = StyleParam.Mirror.AllTicks,
        ShowGrid = false,
        Ticks = StyleParam.TickOptions.Inside
    )

let mirroredLogYAxis = 
    LinearAxis.init(
        Title = Title.init(Text="Texto Eje Y"),
        AxisType = StyleParam.AxisType.Auto,
        ShowLine = true,
        Mirror = StyleParam.Mirror.AllTicks,
        ShowGrid = false,
        Ticks = StyleParam.TickOptions.Inside
    )

let chart_layout = 
    let tmp = Layout()
    tmp?title<- "Titulo Grafica"
    tmp?xaxis<- mirroredXAxis
    tmp?yaxis<- mirroredLogYAxis
    tmp

let colors = x2 |> Array.map (fun name -> match name with
                                                                |"A" -> Color.fromString "green"
                                                                |"B" -> Color.fromString "blue"
                                                                |"C" -> Color.fromString "red"
                                                                |"D" -> Color.fromString "brown"
                                                                |_ -> Color.fromString "deeppink") |> Color.fromColors

let markers = Marker.init(Color=colors)

// Creamos el grafico
Chart.Column(y,x)
    |> Chart.withLineStyle(Width=2.,Dash=StyleParam.DrawingStyle.Dot)
    |> Chart.withLayout chart_layout
    |> Chart.withMarker markers
```

Antes de graficar, debemos agrupar nuestra informacion, para ello usaremos el metodo `groupRowsUsing` del modulo `Frame` el cual recibe una lambda donde el primer elemento es el indice y el segundo elemento son los elementos de nuestro dataframe.

```fsharp

// Ejemplo
let data = df |> Frame.groupRowsUsing(fun selector frame -> frame.GetAs<Type>("Columna"))
   |> Frame.nest
   |> Series.observations
   |> Seq.map (fun ((m, y), frame) ->
        // operacion
   |> Seq.toList

```

Una vez agrupados, aplicamos el metodo `nest` del modulo `Frame` para crear una tupla entre el nuevo indice de la agrupacion y los valores que pertenecen a ese conjunto de datos. Posteriormente el metodo `observations` del modulo `Series` convertimos el resultado en `key` - `value`.
Finalmente con el modulo Seq recorremos el arreglo y armamos la salida para finalmente convertirlo a lista.

In [10]:
// Pasando un tipo de dato a otro tipo
let dts = df.GetColumn<string>("Fecha").Values |> Seq.map (fun x -> DateTime.Parse x)
df?Fecha <- dts

In [11]:
// Funcion para obtener el nombre del mes
let getMonthName (month:int) =
    CultureInfo.CurrentCulture.DateTimeFormat.GetMonthName month

// Type para darle mejor salida a la informacion
type MesVenta(mes, venta) =
    member this.Mes = mes
    member this.Venta = venta
    
// Agrupando


let data = df |> Frame.groupRowsUsing(fun _ c -> c.GetAs<DateTime>("Fecha").Month |> getMonthName)
        |> Frame.nest
        |> Series.observations
        |> Seq.map (fun (m, frame) ->
            // m,
            MesVenta(m, frame.["Ingreso"] |> Stats.sum)
            )
        |> Seq.toList

display(data)

index,Mes,Venta
0,agosto,218232
1,octubre,246142
2,septiembre,238412
3,febrero,167106
4,mayo,199591
5,julio,229188
6,noviembre,204031
7,abril,195132
8,marzo,213630
9,diciembre,237670


In [ ]:
// Graficando
open Plotly.NET
open Plotly.NET.LayoutObjects
open Plotly.NET.TraceObjects

// Obteniendo los valores
let x = data |> List.map (fun x -> x.Mes) |> List.toArray 
let y = data |> List.map (fun x -> x.Venta) |> List.toArray

let mirroredXAxis =
    LinearAxis.init(
        Title = Title.init(Text="Meses"),
        ShowLine = true,
        Mirror = StyleParam.Mirror.AllTicks,
        ShowGrid = false,
        Ticks = StyleParam.TickOptions.Inside
    )

let mirroredLogYAxis = 
    LinearAxis.init(
        Title = Title.init(Text="Ingreso"),
        AxisType = StyleParam.AxisType.Auto,
        ShowLine = true,
        Mirror = StyleParam.Mirror.AllTicks,
        ShowGrid = false,
        Ticks = StyleParam.TickOptions.Inside
    )

let chart_layout = 
    let tmp = Layout()
    tmp?title<- "Ventas Mensuales"
    tmp?xaxis<- mirroredXAxis
    tmp?yaxis<- mirroredLogYAxis
    tmp

Chart.Line(
        x,y,
        ShowMarkers=true,
        MarkerSymbol=StyleParam.MarkerSymbol.Square)
    |> Chart.withLineStyle(Width=2.,Dash=StyleParam.DrawingStyle.Dot)
    |> Chart.withLayout chart_layout

## Finalmente , nuestro ultimo requerimiento consiste en graficar el comportamiento de sus ventas en las diferentes plataformas donde vende sus productos la empresa.

Para ello agruparemos sobre la columna: **Plataforma**

In [24]:
type PlataformaIngreso(plataforma, total)=
    member this.Plataforma=plataforma
    member this.Total=total

let data2 = df |> Frame.groupRowsUsing(fun _ c -> c.GetAs<string>("Plataforma"))
            |> Frame.nest
            |> Series.observations
            |> Seq.map (fun (m, frame) -> PlataformaIngreso(m, frame.["Ingreso"] |> Stats.sum))
            |> Seq.toList
display(data2)

index,Plataforma,Total
0,Website,347315
1,Facebook,396073
2,Instagram,1378842
3,Youtube,449620


In [ ]:
open Plotly.NET.TraceObjects

let x2 = data2 |> List.map (fun x -> x.Plataforma) |> List.toArray 
let y2 = data2 |> List.map (fun x -> x.Total) |> List.toArray 

let mirroredXAxis2 =
    LinearAxis.init(
        Title = Title.init(Text="Plataforma"),
        ShowLine = true,
        Mirror = StyleParam.Mirror.AllTicks,
        ShowGrid = false,
        Ticks = StyleParam.TickOptions.Inside
    )

let mirroredLogYAxis2 = 
    LinearAxis.init(
        Title = Title.init(Text="Monto"),
        AxisType = StyleParam.AxisType.Auto,
        ShowLine = true,
        Mirror = StyleParam.Mirror.AllTicks,
        ShowGrid = false,
        Ticks = StyleParam.TickOptions.Inside
    )

let chart_layout = 
    let tmp = Layout()
    tmp?title<- "Ventas x Plataforma"
    tmp?xaxis<- mirroredXAxis2
    tmp?yaxis<- mirroredLogYAxis2
    tmp

let colors = x2 |> Array.map (fun name -> match name with
                                                                |"Website" -> Color.fromString "green"
                                                                |"Facebook" -> Color.fromString "blue"
                                                                |"Youtube" -> Color.fromString "red"
                                                                |_ -> Color.fromString "deeppink") |> Color.fromColors

let markers = Marker.init(Color=colors)


Chart.Column(y2,x2)
    |> Chart.withLineStyle(Width=2.,Dash=StyleParam.DrawingStyle.Dot)
    |> Chart.withLayout chart_layout
    |> Chart.withMarker markers

# Antes de terminar

Plotly tiene una gran variedad de opciones para graficar, incluso nos da la opcion de utilizar mapas para plasmar nuestros datos al estilo de Folium.
A continuacion se muestra un ejemplo basico para comenzar a trabajar con mapas y como podemos usarlo en combinacion de un API.

In [ ]:
open System.IO
open System.Text
open FSharp.Data
open Newtonsoft.Json

let data = 
    Http.RequestString "https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv"
    |> fun csv -> Frame.ReadCsvString(csv,true,separators=",",schema="fips=string,unemp=float")

let geoJson = 
    Http.RequestString "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
    |> JsonConvert.DeserializeObject 

let locationsGeoJSON: string [] = 
    data
    |> Frame.getCol "fips"
    |> Series.values
    |> Array.ofSeq
let zGeoJSON: int [] = 
    data
    |> Frame.getCol "unemp"
    |> Series.values
    |> Array.ofSeq

Chart.ChoroplethMap(
    locations = locationsGeoJSON,
    z = zGeoJSON,
    Locationmode=StyleParam.LocationFormat.GeoJson_Id,
    GeoJson = geoJson,
    FeatureIdKey="id"
)

|> Chart.withGeo(
    Geo.init(
        Scope=StyleParam.GeoScope.NorthAmerica, 
        Projection=GeoProjection.init(StyleParam.GeoProjectionType.AzimuthalEqualArea),
        ShowLand=true,
        LandColor = Color.fromString "lightgrey"
    )
)
|> Chart.withSize (800.,800.)